In [70]:
import pandas as pd
import numpy as np

In [71]:
titanic = pd.read_csv("../input/titanic/train.csv",index_col="PassengerId")
df = titanic.copy()
df.head(3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [72]:
common_attributes = ["Survived","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
df = df[common_attributes]
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S


In [73]:
df.Sex = np.where(df.Sex.str.contains("female"),1,0)
df.tail(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
889,0,3,1,NaN,1,2,23.45,S
890,1,1,0,26.0,0,0,30.00,C
891,0,3,0,32.0,0,0,7.75,Q


In [74]:
df.Embarked.replace(["C","Q","S"],[0,1,2],inplace=True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,0,22.0,1,0,7.2500,2.0
2,1,1,1,38.0,1,0,71.2833,0.0
3,1,3,1,26.0,0,0,7.9250,2.0


In [75]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [76]:
n_df = np.array(df)

In [77]:
!pip install ycimpute

In [78]:
from ycimpute.imputer import EM
dff = EM().complete(n_df)
dff = pd.DataFrame(dff, columns = common_attributes)

In [ ]:
from ycimpute.imputer import iterforest
dff = iterforest.IterImput().complete(n_df)
dff = pd.DataFrame(dff, columns = common_attributes)

In [79]:
dff.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [80]:
dff.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,3.0,0.0,22.0,1.0,0.0,7.2500,2.0
1,1.0,1.0,1.0,38.0,1.0,0.0,71.2833,0.0
2,1.0,3.0,1.0,26.0,0.0,0.0,7.9250,2.0


In [81]:
from sklearn import preprocessing 
scaler = preprocessing.MinMaxScaler(feature_range = (0,1))

In [82]:
dfff = scaler.fit_transform(dff)
dfff = pd.DataFrame(dfff,columns=common_attributes)
dfff.index = df.index
dfff.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0.0,1.0,0.0,0.275,0.125,0.0,0.014151,1.0
2,1.0,0.0,1.0,0.475,0.125,0.0,0.139136,0.0
3,1.0,1.0,1.0,0.325,0.000,0.0,0.015469,1.0


In [88]:
dfff = dfff.sample(frac=1).reset_index()

In [89]:
dfff.head(3)

,index,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,405,1.0,0.0,1.0,0.450,0.0,0.333333,0.138583,1.0
1,584,0.0,1.0,0.0,0.425,0.0,0.000000,0.015713,1.0
2,664,0.0,1.0,0.0,0.325,0.0,0.000000,0.015713,1.0


In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfff.iloc[:,1:], dfff.Survived, test_size=0.2, random_state=42)

In [84]:
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [85]:
from sklearn.metrics import accuracy_score

y_pred = svm_model.predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.7988826815642458
